In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import random

In [60]:
datas = pd.read_csv("../../too_big_datas/credit_card/creditcard.csv")

In [61]:
display(datas.head())
print("The shape of the datas is {}".format(datas.shape))

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


The shape of the datas is (284807, 31)


In [62]:
fraud_datas = datas[datas.Class==1]
normal_datas = datas[datas.Class==0]
sample_normal = normal_datas.sample(10000)

print("Fraud datas: {}".format(fraud_datas.shape))
print("Normal datas: {}".format(normal_datas.shape))
print("Selected datas: {}".format(selected_datas.shape))

sample_dataset = pd.concat([fraud_datas, sample_normal])

Fraud datas: (492, 31)
Normal datas: (284315, 31)
Selected datas: (10000, 31)


### Content
<ul>
    <li><b>Time:</b> Number of seconds elapsed between the first transaction and this transaction.</li>
    <li><b>V1-28:</b> Input variables after PCA transformation. (PCA = Mathematic transformation that cannot be reversed to orthogonalize datas. Its role here is to provide security by respecting the privacy of the cardholders. But no information is lost in this transformation.)</li>
    <li><b>Amount:</b> The amount of the transaction.</li>
    <li><b>Class:</b> It is the label of the datas. 1 = case of fraud, 0 = otherwise.</li>
</ul>
This dataset contains transactions that occured in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

In [63]:
counts = pd.value_counts(datas["Class"], ascending=True)
print("The number of Fraudulent transactions is: {}".format(counts[1]))
print("The number of Normal transactions is: {}".format(counts[0]))
print("The ratio is: {:.3f}%".format(counts[1]/counts[0]*100))

The number of Fraudulent transactions is: 492
The number of Normal transactions is: 284315
The ratio is: 0.173%


Because the number of positive datas (fraude) is very small compared to the number of negative datas (normal), I should use a unsupervised learning algorithm. 

If the number of positive was approximately equal to the number of negative examples, we could use a classification algorithm because the algorithm could get a sense of what is a fraudulent case and why it is a fraud. But here this is not the case so we should do the contrary. We are going to learn what is a normal case of usage. Then, if the features are widely choosen, the algorithm will be able to detect when a use case is not normal.

In [64]:
v_features = datas.ix[:,1:29].columns

C:\Users\roland\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


For this algorithm to work well, we need to choose/pre-process the features as follow:
<ul>
    <li>It needs to have a (semi) gaussian distribution</li>
    <li>p(x) needs to be large for normal and small for fraud.</li>
    <li>It is preferable that the feature has very large or very small values for anomalies.</li>
</ul>

In [65]:
plt.figure(figsize=(12,28*4))
gs = gridspec.GridSpec(28, 1)
for i, cn in enumerate(datas[v_features]):
    ax = plt.subplot(gs[i])
    sns.distplot(datas[cn][datas.Class == 1], bins=50)
    sns.distplot(datas[cn][datas.Class == 0], bins=50)
    ax.set_xlabel('')
    ax.set_title('histogram of feature: ' + str(cn))
#plt.show()

In [66]:
indexes = datas.columns[1:11]
print(indexes)

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10'], dtype='object')


In [67]:
#sns.pairplot(sample_dataset, vars=indexes , hue="Class")
#plt.show()